In [2]:
from selenium import webdriver
import pandas as pd
import time
from tqdm import tqdm
import re

# 멜론 차트 크롤링

In [81]:
driver=webdriver.Chrome('/Users/gunwook/Desktop/chromedriver')

In [17]:
driver.get('https://www.melon.com/chart/search/index.htm')

In [234]:
chart_1 = driver.find_elements_by_css_selector('#lst50')

In [214]:
singer = []
sing = []
like = []
album = []

In [235]:
# 1위에서 50위까지
for i in range(0,50):
    a = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank02 > a')[0].text
    b = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div.ellipsis.rank01 > span > strong > a')[0].text
    c = int(chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(5) > div > button > span.cnt')[0].text.replace(',',''))
    d = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank03 > a')[0].text
    singer.append(a)
    sing.append(b)
    like.append(c)
    album.append(d)

In [236]:
# chart_2로 가기
driver.find_elements_by_css_selector('#frm > div.paginate.chart_page > span > a')[0].click()

In [237]:
chart_2 = driver.find_elements_by_css_selector('#lst100')

In [238]:
# 51위부터 100위까지
for i in range(0,50):
    a = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank02 > a')[0].text
    b = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div.ellipsis.rank01 > span > strong > a')[0].text
    c = int(chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(5) > div > button > span.cnt')[0].text.replace(',',''))
    d = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank03 > a')[0].text
    singer.append(a)
    sing.append(b)
    like.append(c)
    album.append(d)

In [240]:
driver.get('https://www.melon.com/search/total/index.htm?q=EXO&section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&linkOrText=T&ipath=srch_form')

# 가온차트 크롤링

In [3]:
driver.get('http://www.gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL')

In [4]:
#chart_week_select > option:nth-child(120)
#chart_week_select > option:nth-child(3)
#chart_week_select > option:nth-child(107)

In [6]:
list1 = []
for i in tqdm(range(107, 2, -1)):
    # 주간 설정(range는 원하는만큼 지정)
    driver.find_elements_by_css_selector('#chart_week_select')[0].click()
    time = driver.find_elements_by_css_selector('#chart_week_select > option:nth-child({0})'.format(i))[0].text
    driver.find_elements_by_css_selector('#chart_week_select > option:nth-child({0})'.format(i))[0].click()
    
    
    rank = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr> td.ranking')
    chart_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.subject > p:nth-child(1)')
    gaon_100_score = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.count > p')
    production_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.production > p.pro')
    dist_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.production > p.dist')
    artist = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.subject > p.singer')
    for a,b,c,d,e,f in zip(chart_top_100,gaon_100_score,production_top_100,dist_top_100,artist,rank):
        
        a = re.split('[(-)]+' ,a.text)[0]
        e = e.text.split('|')[0]
        if e == '(여자)아이들':
            pass
        else:
            e = re.split('[(-)]+' ,e)[0]
        b = int(b.text.replace(',',''))
        list1.append([a, c.text, d.text, e, time, b, f.text])
        

100%|██████████| 105/105 [48:43<00:00, 27.85s/it]


In [7]:
df = pd.DataFrame(list1, columns = ['name','production','distributor','artist','time','score','rank'])

In [12]:
df.to_csv('가온차트.csv')

# 지니차트 크롤링

In [82]:
driver.get('https://www.genie.co.kr')

In [14]:
genre = []
runtime = []
sex = []
active_type = []

In [15]:
list2 = list(set(list(df['artist'] + ',' + df['name'])))

In [27]:
len(list2)

1454

In [28]:
df['name_artist'] = df['artist'] + ',' + df['name']

In [17]:
# 노래 정보 뽑아오기
# 장르, 런타임, 성별, 활동유형
for i in tqdm(set(list2)):
    try:
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        if ':' in time:
            pass
        else:
            time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value').text
            gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        runtime.append(time)
        genre.append(gen)
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_elements_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.info > a.artist.ellipsis')[0].click()
        activity_type = driver.find_elements_by_css_selector('#body-content > div.artist-main-infos > div.info-zone > ul > li:nth-child(1) > span.value')[0].text.split('/')
        sex.append(activity_type[0])
        active_type.append(activity_type[1])
    except:
        i = i.split(',')
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        if ':' in time:
            pass
        else:
            time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value').text
            gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        genre.append(gen)
        runtime.append(time)
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_elements_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.info > a.artist.ellipsis')[0].click()
        activity_type = driver.find_elements_by_css_selector('#body-content > div.artist-main-infos > div.info-zone > ul > li:nth-child(1) > span.value')[0].text.split('/')
        sex.append(activity_type[0])
        active_type.append(activity_type[1])


100%|██████████| 1454/1454 [1:04:54<00:00,  2.68s/it]


In [22]:
data1 = pd.DataFrame(genre,columns = ['장르'])

In [32]:
data = pd.DataFrame(list2,columns = ['조인'])

In [30]:
data2 = pd.DataFrame(runtime,columns = ['런타임'])
data3 = pd.DataFrame(sex,columns = ['성별'])
data4 = pd.DataFrame(active_type,columns = ['활동유형'])

In [34]:
final = pd.concat([data,data1,data2,data3,data4],axis= 1)

In [36]:
final = final.rename(columns = {'조인':'name_artist'})

In [38]:
df = pd.merge(df,final,how='left',on='name_artist')

In [39]:
df.to_csv('최종.csv')

In [7]:
df = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/최종.csv')

In [10]:
del df['Unnamed: 0']

In [13]:
import re
p = re.compile('[OST]')

In [16]:
list1 = []
for i in df['장르']:
    a = p.search(i)
    list1.append(a)

In [32]:
temp = df[df['장르'].str.contains('OST')]['name_artist'].unique()

In [34]:
temp = list(temp)

In [63]:
temp

['이수,My Way',
 '폴킴,모든 날, 모든 순간 ',
 '문문 ,결혼',
 '멜로망스,짙어져',
 '에일리 ,첫눈처럼 너에게 가겠다',
 'Sondia,어른',
 '양요섭,길에서',
 '김나영,그대만 보여서',
 '박효신,그 날 ',
 '이수현,소리',
 '멜로망스,좋은 날',
 '세븐틴,A-TEEN',
 '박원,이방인',
 '로이킴,왜 몰랐을까',
 '벤 ,미워도 다시 한 번',
 '뉴이스트 W,AND I',
 '백지영,See You Again ',
 '정승환,잘 지내요',
 '거미,지워져',
 '모트 ,도망가지마',
 '세정,정인 ',
 '10cm,Perfect',
 '벤 ,If You Were Me',
 '황치열,어찌 잊으오',
 '김민석,너에게',
 '첸 ,벚꽃연가',
 '다비치,꿈처럼 내린',
 '케이윌 ,내 생에 아름다운',
 '로시 ,구름',
 '웬디,Goodbye',
 '볼빨간사춘기,My Trouble',
 'Queen,Bohemian Rhapsody ',
 'Queen,Bohemian Rhapsody',
 'Queen,Somebody To Love',
 '폴킴,사랑 알 수 없나봐',
 '에일리,Is You',
 '하진,We all lie',
 'Queen,Radio Ga Ga ',
 '정승환,너였다면',
 '크러쉬 ,Beautiful',
 '에이핑크 ,%%',
 '양다일,I`m Here',
 '백아연,그대여야만 해요',
 '허각,Cosmos',
 '첸 ,Make it count',
 '잔나비,나는 볼 수 없던 이야기',
 '로시 ,레인보우 ',
 'Post Malone & Swae Lee,Sunflower ',
 '성시경,니 곁이라면',
 '웬디,What If Love',
 '김나영,너의 모든 기억속에',
 '로꼬 ,우연히 봄',
 '백예린,스며들기 좋은 오늘',
 '세븐틴,9-TEEN',
 'Naomi Scott,Speechless ',
 'Mena Massoud, Naomi Scott,A Whole New World',
 '방탄소년단

In [39]:
drama = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(2) > span.value > a').text

In [77]:
p = re.compile('.+(?=OST)')
b = p.search(drama).strip()

In [79]:
b.group().strip()

'미스터 션샤인'

In [55]:
(b.group())

'돈꽃'

In [112]:
drama

'OST / 해외영화'

In [ ]:
#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value > a

In [94]:
drama = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text

In [ ]:
#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(2) > span.value > a

In [104]:
elem = driver.find_element_by_id('sc-fd')
elem.clear()
elem.send_keys(i)


In [113]:
i

['백지영', 'See You Again ']

In [121]:
i

'로꼬 ,우연히 봄'

In [123]:
drama_list

['돈꽃',
 "'키스 먼저 할까요?'",
 '이번 생은 처음이라',
 'Yellow',
 '도깨비',
 '나의 아저씨',
 '리플리 Vol.1',
 '김비서가 왜 그럴까',
 '미스터 션샤인',
 '미스터 션샤인',
 '미스터 션샤인',
 'A-TEEN Part.3',
 '미스터 션샤인',
 '아는 와이프',
 '리플리 Vol.3',
 '미스터 션샤인',
 '미스터 션샤인',
 '라이프',
 '백일의 낭군님',
 'A-TEEN Part.4',
 '미스터 션샤인',
 '연애플레이리스트 3 Part.1 (웹드라마)',
 '미스터 션샤인',
 '미스터 션샤인',
 '연애플레이리스트 3 Part.2 (웹드라마)',
 '백일의 낭군님',
 '뷰티 인사이드',
 '뷰티 인사이드',
 '뷰티 인사이드',
 '뷰티 인사이드',
 'WHY : 당신이 연인에게 차인 진짜 이유',
 '영화 보헤미안 랩소디 OST (Bohemian Rhapsody',
 '영화 보헤미안 랩소디 OST (Bohemian Rhapsody',
 '영화 보헤미안 랩소디 OST (Bohemian Rhapsody',
 'WHY : 당신이 연인에게 차인 진짜 이유',
 '알함브라 궁전의 추억',
 'SKY 캐슬',
 '영화 보헤미안 랩소디 OST (Bohemian Rhapsody',
 '또 오해영',
 '도깨비',
 '영혼수선공',
 '알함브라 궁전의 추억',
 '남자친구',
 '일단 뜨겁게 청소하라',
 '진심이 닿다',
 '로맨스는 별책부록',
 '로맨스는 별책부록',
 '스파이더맨 : 뉴 유니버스 OST (Spider-Man : Into The Spider-Verse',
 '왕이 된 남자',
 '진심이 닿다',
 '로맨스는 별책부록',
 '냄새를 보는 소녀',
 'A-TEEN 2 Part.1',
 'A-TEEN 2 Part.2',
 'Aladdin (Original Motion Picture Soundtrack)',
 'Aladdin (Original Motion Picture Soundtrack)',


In [131]:
temp

['이수,My Way',
 '폴킴,모든 날, 모든 순간 ',
 '문문 ,결혼',
 '멜로망스,짙어져',
 '에일리 ,첫눈처럼 너에게 가겠다',
 'Sondia,어른',
 '양요섭,길에서',
 '김나영,그대만 보여서',
 '박효신,그 날 ',
 '이수현,소리',
 '멜로망스,좋은 날',
 '세븐틴,A-TEEN',
 '박원,이방인',
 '로이킴,왜 몰랐을까',
 '벤 ,미워도 다시 한 번',
 '뉴이스트 W,AND I',
 '백지영,See You Again ',
 '정승환,잘 지내요',
 '거미,지워져',
 '모트 ,도망가지마',
 '세정,정인 ',
 '10cm,Perfect',
 '벤 ,If You Were Me',
 '황치열,어찌 잊으오',
 '김민석,너에게',
 '첸 ,벚꽃연가',
 '다비치,꿈처럼 내린',
 '케이윌 ,내 생에 아름다운',
 '로시 ,구름',
 '웬디,Goodbye',
 '볼빨간사춘기,My Trouble',
 'Queen,Bohemian Rhapsody ',
 'Queen,Bohemian Rhapsody',
 'Queen,Somebody To Love',
 '폴킴,사랑 알 수 없나봐',
 '에일리,Is You',
 '하진,We all lie',
 'Queen,Radio Ga Ga ',
 '정승환,너였다면',
 '크러쉬 ,Beautiful',
 '에이핑크 ,%%',
 '양다일,I`m Here',
 '백아연,그대여야만 해요',
 '허각,Cosmos',
 '첸 ,Make it count',
 '잔나비,나는 볼 수 없던 이야기',
 '로시 ,레인보우 ',
 'Post Malone & Swae Lee,Sunflower ',
 '성시경,니 곁이라면',
 '웬디,What If Love',
 '김나영,너의 모든 기억속에',
 '로꼬 ,우연히 봄',
 '백예린,스며들기 좋은 오늘',
 '세븐틴,9-TEEN',
 'Naomi Scott,Speechless ',
 'Mena Massoud, Naomi Scott,A Whole New World',
 '방탄소년단

In [129]:
drama_list = []
for i in tqdm(temp):
    try:
        if i == '로꼬 ,우연히 봄':
            i = '우연히 봄'
        if i == 'Kristen Bell, Idina Menzel, Josh Gad, Jonathan Groff, Cast of Frozen 2Frozen 2 ,Some Things Never Change':
            i = 'Frozen 2 ,Some Things Never Change'
        if i == '크러쉬 ,둘만의 세상으로 가':
            i = 'Crush ,둘만의 세상으로 가'
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        drama = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(2) > span.value').text
        if 'OST' in drama:
            drama_list.append(p.search(drama).group().strip())
        else:
            drama_list.append(drama)
    except:
        if i == '로꼬 ,우연히 봄':
            i = '우연히 봄'
        if i == 'Kristen Bell, Idina Menzel, Josh Gad, Jonathan Groff, Cast of Frozen 2Frozen 2 ,Some Things Never Change':
            i = 'Frozen 2 ,Some Things Never Change'
        if i == '크러쉬 ,둘만의 세상으로 가':
            i = 'Crush ,둘만의 세상으로 가'
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        drama = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        if 'OST' in drama:
            drama_list.append(p.search(drama).group().strip())
        else:
            drama_list.append(drama)
    

 97%|█████████▋| 146/150 [05:16<00:08,  2.17s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a"}
  (Session info: chrome=81.0.4044.138)


In [133]:
drama_list.append('X by X ［결핍]')

In [136]:
drama_list.append('더 킹 : 영원의 군주')

In [139]:
df_fin = pd.DataFrame(temp,columns=['artist'])

In [140]:
df_fin['제목'] = drama_list

In [142]:
df_fin.to_csv('드라마 리스트.csv')

# 멜론 좋아요 뽑기

In [35]:
driver.get('https://www.melon.com/search/total/index.htm?q=%EC%82%AC%EB%9E%91%EC%97%90+%EC%97%B0%EC%8A%B5%EC%9D%B4+%EC%9E%88%EC%97%88%EB%8B%A4%EB%A9%B4&section=&linkOrText=T&ipath=srch_form')

In [36]:
likes = []

In [31]:
# 중간에 계속 팅기는 문제 발생. 
# 시간 텀 줘도 계속 팅기는 문제 생김
count = 0
for i in tqdm(list1):
    try:
        elem = driver.find_element_by_id('top_search')
        elem.clear()
        elem.send_keys(i)
        driver.find_element_by_css_selector('#header_wrap > div.wrap_search_field > fieldset > button.btn_icon.search_m > span').click()
        like = int(driver.find_element_by_css_selector('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > button > span.cnt').text.replace(',',''))
        likes.append(like)
        if count == 5:
            time.sleep(10)
            count = 0
        else:
            count += 1
    except:
        i = i.split(',')
        elem = driver.find_element_by_id('top_search')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_element_by_css_selector('#header_wrap > div.wrap_search_field > fieldset > button.btn_icon.search_m > span').click()
        like = int(driver.find_element_by_css_selector('#frm_searchSong > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > button > span.cnt').text.replace(',',''))
        likes.append(like)
        if count == 5:
            time.sleep(10)
            count = 0
        else:
            count += 1




  0%|          | 0/689 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="top_search"]"}
  (Session info: chrome=81.0.4044.129)
